In [63]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import random
import csv
import os
from PIL import Image

from Config import Config
from Models import PreTrainedKPD, PreTrainedConf, Tools

In [64]:
"""
Load the model weights.
"""

device = "mps" if torch.backends.mps.is_available() else "cpu"

kpd_model = PreTrainedKPD.KeypointDetectionModel().to(device)
kpd_model.load_state_dict(torch.load("Models/Saves/pretrainedKPD_weights00166.pth", map_location=device))

conf_model = PreTrainedConf.ConfidenceModel().to(device)
conf_model.load_state_dict(torch.load("Models/Saves/pretrainedConf_weights00166.pth", map_location=device))

/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_58269/923896920.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/mai

<All keys matched successfully>

In [84]:
notes = pd.read_csv('CSVs/combined.csv')
preds = pd.read_csv('predictions_17december.csv')

preds['Image Name'] = preds['Image Name'].map(lambda s : os.path.splitext(s)[0])
notes['Image Name'] = notes['Image Name'].map(lambda s : os.path.splitext(s)[0])

notes = notes.groupby(['Image Name']).mean()

In [88]:
preds.set_index('Image Name').join(notes, how='inner', lsuffix='_predicted', rsuffix='_real')

,x1_predicted,y1_predicted,x2_predicted,y2_predicted,Error estimate,x1_real,y1_real,x2_real,y2_real
Image Name,,,,,,,,,
antweb1038578_p_1,968.425214,278.650289,888.545968,303.143567,0.342074,703.0,334.0,1130.0,426.0
casent0003131_p_1,732.926435,206.051771,500.225782,190.303367,0.138038,349.0,223.0,784.0,241.0
casent0217464_p_1,760.971606,264.559571,366.700752,164.620090,0.271314,407.0,128.0,751.0,253.0
casent0103447_p_1,655.816242,194.425754,449.774630,201.630753,0.329780,270.0,216.0,628.0,313.0
casent0249764_p_1,601.015129,234.377738,373.912095,196.411209,0.390359,359.5,241.0,641.0,310.5
...,...,...,...,...,...,...,...,...,...
casent0907340_p_1,671.910096,177.615879,482.972262,203.188506,0.322776,227.0,205.0,849.0,356.0
casent0913305_p_1,675.294641,236.662282,341.467381,128.393289,0.372602,299.0,121.0,704.0,267.0
casent0178037_p_1,557.770351,209.588442,320.066273,187.612372,0.262641,253.0,186.0,575.0,247.0


In [67]:
def get_image_shape(file_path):
    with Image.open(file_path) as img:
        return img.size


notes['shape'] = notes.index.map(lambda s : get_image_shape(
    os.path.join(Config.images_folder_path, s + '.png')
    )
)

In [68]:
errors = (keypoints - outputs)**2
mse_per_vector = torch.mean(errors, dim=1, keepdim=True)

mse_per_vector = torch.clamp(mse_per_vector, max=clamp_threshold)
mse_per_vector_np = mse_per_vector.cpu().numpy()

# Step 1: Normalize by mean and variance
mean = mse_per_vector.mean()
std = mse_per_vector.std()
normalized_vector = (mse_per_vector - mean) / std

# Step 2: Scale to range [0, 1]
min_val = normalized_vector.min()
max_val = normalized_vector.max()
scaled_errors = (normalized_vector - min_val) / (max_val - min_val)

NameError: name 'keypoints' is not defined